In [ ]:
import pandas as pd
import glob
import yaml
import os

In [ ]:
# Load settings from config.yaml
with open("config.yaml", 'r') as file:
    config = yaml.safe_load(file)

In [ ]:
monotonicity_score_weight = config['ideal_weights']['parameters']['monotonicity_score_weight']
std_deviation_weight = config['ideal_weights']['parameters']['std_deviation_weight']
avg_proportional_increase_weight = config['ideal_weights']['parameters']['avg_proportional_increase_weight']

In [ ]:
# Load the ideal votable supply data
def load_ideal_data(file_paths):
    ideal_dfs = []
    for file_path in file_paths:
        ideal_df = pd.read_csv(file_path)
        ideal_df['Date'] = pd.to_datetime(ideal_df['Date'], errors='coerce')
        ideal_df = ideal_df.dropna(subset=['Date'])
        ideal_dfs.append(ideal_df)
    return ideal_dfs

# Calculate performance metrics
def calculate_metrics(ideal_dfs):
    results = []
    for data in ideal_dfs:
        if 'weights' not in data.columns:
            raise KeyError("Weights column is missing from the data.")

        # Calculate the proportional increase
        data['Pincrease'] = data['IVS'] / data['Votable Supply']

        # Calculate the monotonicity score, average proportional increase, and standard deviation
        monotonicity_score = (data['Pincrease'].diff().fillna(0) >= 0).mean()
        avg_proportional_increase = data['Pincrease'].mean()
        std_deviation = data['Pincrease'].std()

        # Calculate the total score
        total_score = (
            monotonicity_score * monotonicity_score_weight
            - std_deviation * std_deviation_weight
            + avg_proportional_increase * avg_proportional_increase_weight
        )

        # Append the results
        results.append({
            'Weights': data['weights'].iloc[0], 
            'Monotonicity Score': monotonicity_score,
            'Average Proportional Increase': avg_proportional_increase,
            'Standard Deviation': std_deviation,
            'Total Score': total_score
        })

    return pd.DataFrame(results)

# Main process
def main():
    weight_combination_folder = config['generate_weight_combinations']['data']['all_weight_combinations_folder_path']
    weight_combination_files = glob.glob(os.path.join(weight_combination_folder, "*.csv"))

    # Step 1: Load data
    ideal_dfs = load_ideal_data(weight_combination_files)

    # Step 2: Calculate metrics
    metrics_df = calculate_metrics(ideal_dfs)

    # Step 3: Rank weight combinations
    metrics_df = metrics_df.sort_values(by='Total Score', ascending=False)

    # Step 4: Output results
    ideal_weights_data_path = config['ideal_weights']['data']['ideal_weights_data_path']
    metrics_df.to_csv(ideal_weights_data_path, index=False)
    print("Data saved successfully")

    # Print the ideal weights
    best_weights = metrics_df.iloc[0]
    print("\nIdeal Weights:")
    print(f"Weights: {best_weights['Weights']}")
    print(f"Monotonicity Score: {best_weights['Monotonicity Score']:.2f}")
    print(f"Average Proportional Increase: {best_weights['Average Proportional Increase']:.2f}")
    print(f"Standard Deviation: {best_weights['Standard Deviation']:.2f}")
    print(f"Total Score: {best_weights['Total Score']:.2f}")

if __name__ == "__main__":
    main()
